In [1]:
%load_ext sql
import os
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
conn = create_engine(dsl)

# MagicコマンドでSQLを書くための設定
%sql conn

In [2]:
%%sql
drop table if exists tbl;
create table tbl (
    id integer primary key,
    value integer not null
);

insert into tbl
values (1,4),
    (2,6),
    (3,7),
    (4,0),
    (5,5),
    (6,10),
    (7,10);

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
7 rows affected.


[]

In [5]:
%%sql
-- percentileを使う
select *
from tbl
where value = (
    select
        percentile_disc(0.5) within group (order by value) as median
    from tbl
)

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


id,value
2,6


In [7]:
%%sql
-- row_numberを使う
select *
from (
    select *,
        row_number() over(order by value asc),
        row_number() over(order by value desc)
    from tbl
) as tmp(id, value, hi, lo)
where hi in (lo-1, lo, lo+1)

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


id,value,hi,lo
2,6,4,4
